In [1]:
from collections import defaultdict
from flask import Flask,request, make_response
from flask_cors import CORS
import pandas as pd
import numpy as np
import scipy
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import time
import json

In [2]:
Ratings = pd.read_csv('data/animelist.csv')
Animes = pd.read_csv('data/anime.csv')

In [3]:

Animes=Animes[['Name','MAL_ID','Score','Type']]
Animes = Animes.rename(columns={'MAL_ID': 'anime_id'})


In [4]:
Ratings= pd.merge(Ratings, Animes, on="anime_id", how="inner")
Ratings = Ratings[Ratings['Type']=='TV']
Ratings = Ratings[['user_id', 'Name','rating']]

In [5]:
counts = Ratings['user_id'].value_counts()
Ratings = Ratings[Ratings['user_id'].isin(counts[counts >= 300].index)]

In [6]:
similar_animes_pivot=Ratings.pivot_table(index='Name',columns='user_id',values='rating').fillna(0)
similar_animes_pivot.head()

user_id,3,12,16,17,19,21,34,41,44,46,...,353357,353358,353365,353370,353381,353383,353385,353390,353395,353398
Name,,,,,,,,,,,,,,,,,,,,,
"""Eikou Naki Tensai-tachi"" Kara no Monogatari",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Roots,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
.hack//Sign,0.0,10.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,0.0
0-sen Hayato,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def get_all_animes():
    return (Ratings.loc[:, ["Name"]].drop_duplicates().to_json(orient="records"))

In [10]:
from scipy.sparse import csr_matrix
#Creating a sparse matrix
anime_matrix = csr_matrix(similar_animes_pivot.values)


from sklearn.neighbors import NearestNeighbors

#Fitting the model
anime_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
anime_knn.fit(anime_matrix)




NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
def recommend(anime_Name):
  distances, indices = anime_knn.kneighbors(similar_animes_pivot.loc[anime_Name,:].values.reshape(1, -1), n_neighbors = 6)
  for i in range(0, len(distances.flatten())):
      if i == 0:
          print('Recommendations for {0}:\n'.format(anime_Name))
      else:
          print('{0}: {1}, with distance of {2}:'.format(i, similar_animes_pivot.index[indices.flatten()[i]], distances.flatten()[i]))


In [12]:
import pickle as pkl

pkl.dump(anime_knn,open("data/anime.pkl",'wb'),pkl.HIGHEST_PROTOCOL)
pkl.dump(similar_animes_pivot,open("data/animes_pivot.pkl",'wb'),pkl.HIGHEST_PROTOCOL)


In [13]:
from operator import itemgetter
from heapq import nsmallest

def generate_recommendations(anime_dict):
  recommend_dict = {}
  for anime in anime_dict:
    distances, indices = anime_knn.kneighbors(similar_animes_pivot.loc[anime,:].values.reshape(1, -1), n_neighbors = 10)

    for i in range(0, len(distances.flatten())):
        if i == 0:
          continue
        else:
            dist=distances.flatten()[i]/anime_dict[anime]
            id=similar_animes_pivot.index[indices.flatten()[i]]
            if not(id in anime_dict.keys() or id in recommend_dict.keys()):
              recommend_dict[id]=dist

  #res = dict(sorted(recommend_dict.items(), key = itemgetter(1))[:10])
  res = nsmallest(10, recommend_dict, key = recommend_dict.get)
  return json.dumps(res)
#find n smallest dists in reccomend_dict and return          

In [ ]:

app = Flask(__name__)
CORS(app)

@app.route('/api/animes', methods=['GET'])
def animes():
    animeList = get_all_animes()
    return animeList


@app.route('/api/recommendations', methods=['POST'])
def recommendations():
    data = request.get_json()
    for currentAnime in data:
        key = currentAnime.get('animeName')
        
    recommendations = generate_recommendations(data)
    return make_response(recommendations)

if __name__ == "__main__":
    app.run(port=5555, debug=True, use_reloader=False)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jul/2022 10:45:34] "POST /api/recommendations HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/nikitaagrawal/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2464, in __call__
    return self.wsgi_app(environ, start_response)
  File "/Users/nikitaagrawal/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2450, in wsgi_app
    response = self.handle_exception(e)
  File "/Users/nikitaagrawal/opt/anaconda3/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/nikitaagrawal/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1867, in handle_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/nikitaagrawal/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/nikitaagrawal/opt/anaconda

127.0.0.1 - - [15/Jul/2022 10:45:52] "GET /api/recommendations?__debugger__=yes&cmd=resource&f=jquery.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2022 10:45:53] "GET /api/recommendations?__debugger__=yes&cmd=resource&f=console.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2022 10:45:53] "GET /api/recommendations?__debugger__=yes&cmd=resource&f=ubuntu.ttf HTTP/1.1" 200 -


In [ ]:
generate_recommendations({'.hack//Tasogare no Udewa Densetsu':10,'Fullmetal Alchemist':10,'.hack//Roots':9,'.hack//Sign':8,'Chrno Crusade':7,'Chobits':8})